In [1]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

### 도구 (Tools)

도구(Tool) 에이전트, 체인 또는 LLM(Large Language Model)이 특정 작업을 수행하거나 외부 데이터에 접근할 수 있도록 돕는 기능을 제공합니다. <br>

Langchain에 통합된 도구를 사용할 수 있으며, 사용자 정의 도구를 사용할 수도 있습니다.<br>

[Langchain에 통합된 도구](https://python.langchain.com/v0.1/docs/integrations/tools/)

In [2]:
from langchain_teddynote import logging

logging.langsmith('PROJECT_2025_LANGCHAIN')

LangSmith 추적을 시작합니다.
[프로젝트명]
PROJECT_2025_LANGCHAIN


In [3]:
from langchain_experimental.tools import PythonAstREPLTool

# 파이썬 코드를 실행하는 도구 생성
python_tool = PythonAstREPLTool()   

In [4]:
answer = python_tool.invoke('print(100 + 200)')     # 코드를 실행 및 결과 리턴
print(answer)

300



###  LLM 에게 파이썬 코드를 작성하도록 요청하고 결과를 리턴

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

from langchain_experimental.tools import PythonAstREPLTool

In [6]:
def print_and_execute(code, debug=True):       # 파이썬 코드를 실행하고 중간 과정을 출력하고 실행결과를 리턴
    python_tool = PythonAstREPLTool()
    
    if debug:
        print('CODE:')
        print(code)

    return python_tool.invoke(code)

In [7]:
prompt = ChatPromptTemplate.from_messages(      # 파이썬 코드 작성을 지시하는 프롬프트
    [
        (
            'system',
            'You are Raymond Hetting, an expert python programmer, well versed in meta-programming and elegant, concise and short but well documented code. You follow the PEP8 style guide. '
            'Return only the code, no intro, no explanation, no chatty, no markdown, no code block, no nothing. Just the code.',
        ),
        ('human', '{input}'),
    ]
)

In [8]:
llm = ChatOpenAI( 
    api_key=key,
    model='gpt-4o-mini', 
    temperature=0
)

In [9]:
output_parser = StrOutputParser()

In [10]:
chain = prompt | llm | output_parser | RunnableLambda(print_and_execute)

In [11]:
print(chain.invoke('로또 번호 생성기를 출력하는 코드를 작성해주세요.', debug=True))

CODE:
import random

def generate_lotto_numbers():
    """Generate a set of 6 unique lotto numbers from 1 to 45."""
    return sorted(random.sample(range(1, 46), 6))

if __name__ == "__main__":
    print(generate_lotto_numbers())



### 검색 API 도구

TavilyAnswer 검색 API를 활용하여 검색 기능을 구현하는 도구. 
<br>

`TavilySearchResults` Tavily 검색 API를 실행하고 JSON 형식의 결과를 리턴.
<br>

매개변수: 

- max_results  (int) : 리턴 할 최대 검색 결과 개수 (default: 5)
- search_depth(str) : 검색 깊이(basic 또는 advanced) 
  - basic : 기본값
  - advanced : Tavily 검색 API가 더 깊고 광범위한 검색을 수행하도록 지시.
- include_domains (List[str]): 검색 결과에 포함할 도메인 목록
- exclude_domains (List[str]): 검색 결과에서 제외할 도메인 목록
- include_answer (bool): 원본 쿼리에 대한 짧은 답변을 포함 할 것인가
- include_raw_content (bool): 각 사이트의 정제된 HTML 콘텐츠를 포함할 것인지. 원본을 포함할 것인가
- include_images (bool): 쿼리 관련 이미지 목록 포함할 것인지

리턴 값:

- 검색 결과를 포함하는 JSON 문자열(uri, content)

In [12]:
from langchain_community.tools.tavily_search import TavilySearchResults

tool = TavilySearchResults(                         # 도구 생성
    max_results=3,                                  # 검색 결과 수
    search_depth='advanced',                        # Tavily 검색 API가 더 깊고 광범위한 검색을 수행하도록 지시
    include_raw_content=True,                       # 검색 결과의 원본 내용 포함
    include_answer=True,                            # 검색 결과를 바탕으로 생성된 답변 포함
    include_images=True,                            # 검색 결과에 이미지 포함
    include_domains=['github.io', 'naver.com']      # 검색 대상 도메인을 github.io와 naver.com으로 제한
)

In [13]:
answer = tool.invoke({'query':'랭체인 Tool에 대해서 알려주세요'})

In [14]:
answer

[{'url': 'https://post.naver.com/viewer/postView.naver?volumeNo=38409843&vType=VERTICAL',
  'content': 'LangChain(랭체인) 은 대규모 언어 모델 (LLM) 을 활용한 애플리케이션 개발에 특화된 오픈소스 프레임워크입니다. LangChain 은 기존 언어 모델의 한계를 극복하고, AI 기술을 활용한 새로운 애플리케이션을 구축할 수 있는 중요한 도구로 자리 잡고 있습니다.'},
 {'url': 'https://m.blog.naver.com/sangwon38383/223183173792',
  'content': '최근 AI에 관한 밋업에서는 랭체인(LangChain) 에 대한 이야기가 꼭 나온다. 그런데 랭체인은 정확히 뭐인가? 한마디로 요약하자면 언어모델을 활용한 개발 작업을 편리하게 할 수 있도록 도와주는 프레임워크이다. 즉 랭체인 없이도 API 또는 오픈소스로 제공되는'},
 {'url': 'https://blog.naver.com/PostView.naver?blogId=beyond-zero&logNo=223386007783',
  'content': 'LangChain은 ChatGPT와 같은 대규모 언어 모델(LLM)의 향상된 기능을 효율적으로 구현하기 위한 라이브러리 이다.. 이 글을 쓰는 시점인 2024년 3월 15일 기준으로 Python, TypeScript기반 라이브러리가 공개되었다. 기본적으로 필자 생각으로는 TypeScript에 비해 Python이 LangChain에서 사용할 수 있는 기능이 많기'}]

In [15]:
from langchain_community.tools.tavily_search import TavilySearchResults

tool = TavilySearchResults(                         # 도구 생성
    max_results=3,                                  # 검색 결과 수
    search_depth='advanced',                        # Tavily 검색 API가 더 깊고 광범위한 검색을 수행하도록 지시
    include_raw_content=True,                       # 검색 결과의 원본 내용 포함
    include_answer=True,                            # 검색 결과를 바탕으로 생성된 답변 포함
    include_images=True,                            # 검색 결과에 이미지 포함
    include_domains=['google.com', 'naver.com']     # 검색 대상 도메인을 github.io와 naver.com으로 제한
)

answer = tool.invoke({'query':'인어공주에 대해서 알려주세요'})
print(answer)

[{'url': 'https://m.blog.naver.com/cstory7/220634790035', 'content': '인어공주에 대해 모르는 사람이 없을꺼라 믿어 의심치 않습니다ㅋㅋ 오늘은 이렇게도 유명한 명작 동화이자 디즈니 프린세스 중 한명인 인어공주에 대해 조금 자세한 이야기를 해볼까해요 이름하야 인어공주 비하인드 스토리!!'}, {'url': 'https://blog.naver.com/PostView.naver?blogId=dalbat_&logNo=223251160300', 'content': '여러분께 인어공주에 대한 이야기를 들려드린다 생각하니 마음이 두근거립니다. 자, 그럼 떠나봅시다! ... 걸을 때마다 극심한 고통을 느낀다는 조건은 인간의 가치에 대해서 말하고 있습니다. 우리는 눈 떠보니 인간으로 태어나 그렇게 살아가고 있습니다'}, {'url': 'https://m.blog.naver.com/dye5h/222198049083', 'content': '디즈니영화 인어공주는 1989년에 개봉한 작품 이에요!. 굉장히 오래된 영화죠~~??ㅎㅎㅎㅎ. 하지만 지금 봐도 재미있는 디즈니영화랍니다^^ 안데르센의 동명 동화를 원작 으로 하고 있는데,. 인어공주가 물거품으로 변하는 새드엔딩의'}]
